In [194]:
from flask import Flask, render_template, request, jsonify
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import json
import time


app = Flask(__name__)
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])
data= json.load(open("new_dict_all_vid_tuple.json", encoding='utf-8'))

In [195]:
def generate_data(data):
    for video in data:
        features=[]
        for feat in video["features"]:
            lieu = [(str(x[0]), str(x[1])) for x in feat["lieu"]]
            objects= [(str(x[0]), str(x[1])) for x in feat["objects"] if feat["objects"] is not None]
            bndbox= [(x[0], x[1], x[2], x[3]) for x in feat["bndbox"]]
            features.append({"frame": feat["frame"], "objects": objects, "bndbox": bndbox, "lieu": lieu})
        yield {
            "_index": "video",
            "_type": "features",
            "_source": {
                "idVideo": video["idVideo"],
                "fps": video["fps"],
                "features": features
            }
        }

if es.indices.exists('video')==True:
    es.indices.delete(index='video')
    bulk(es, generate_data(data))
else :
    bulk(es, generate_data(data))

C:\Users\vgvic\AppData\Local\Temp\ipykernel_2036\3265386875.py:19: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists('video')==True:


In [196]:
query = { "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "should": [
              {
                "match_phrase": {
                  "features.lieu": "bar"
                }
              }
            ],
            "minimum_should_match": 1
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}
result = es.search(index = "video", body =query)
videos = [elt['_source'] for elt in result["hits"]["hits"]]

C:\Users\vgvic\AppData\Local\Temp\ipykernel_2036\1812455906.py:23: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index = "video", body =query)


In [197]:
videos

[{'idVideo': 'Murder Mystery 2 _ Bande-annonce officielle VF _ Netflix France-p0OljkLzV3w.mp4',
  'fps': 24,
  'features': [{'frame': 1,
    'objects': [],
    'bndbox': [],
    'lieu': [['tower', '0.30456575751304626'],
     ['skyscraper', '0.08605102449655533'],
     ['arena/performance', '0.05752118304371834'],
     ['fountain', '0.05442429333925247'],
     ['industrial_area', '0.02278660237789154'],
     ['mosque/outdoor', '0.02216089516878128'],
     ['amusement_park', '0.020923366770148277'],
     ['downtown', '0.02052483707666397'],
     ['sky', '0.020047027617692947'],
     ['castle', '0.01568487286567688']]},
   {'frame': 2,
    'objects': [],
    'bndbox': [],
    'lieu': [['locker_room', '0.07338766008615494'],
     ['canal/urban', '0.04131084680557251'],
     ['fountain', '0.03747288137674332'],
     ['plaza', '0.036764536052942276'],
     ['ice_skating_rink/outdoor', '0.03634338825941086'],
     ['embassy', '0.03543225675821304'],
     ['arena/performance', '0.028871897608

In [198]:
for video in videos : 
    for frame in video['features']:
        for lieu in frame['lieu']:
            if 'bar' in lieu[0]:
                print(video['idVideo'])
                print(frame['frame'])
                print(lieu)

Murder Mystery 2 _ Bande-annonce officielle VF _ Netflix France-p0OljkLzV3w.mp4
38
['sushi_bar', '0.026281563565135002']
Murder Mystery 2 _ Bande-annonce officielle VF _ Netflix France-p0OljkLzV3w.mp4
38
['bar', '0.010339248925447464']
Murder Mystery 2 _ Bande-annonce officielle VF _ Netflix France-p0OljkLzV3w.mp4
68
['barndoor', '0.011953943409025669']
Sam Smith, Kim Petras - Unholy (Official Music Video)-Uq9gPaIzbe8.mp4
29
['barndoor', '0.020870307460427284']
Sam Smith, Kim Petras - Unholy (Official Music Video)-Uq9gPaIzbe8.mp4
51
['barndoor', '0.017935732379555702']
Sam Smith, Kim Petras - Unholy (Official Music Video)-Uq9gPaIzbe8.mp4
64
['barndoor', '0.01559110451489687']
Sam Smith, Kim Petras - Unholy (Official Music Video)-Uq9gPaIzbe8.mp4
83
['bar', '0.012275643646717072']
Sam Smith, Kim Petras - Unholy (Official Music Video)-Uq9gPaIzbe8.mp4
90
['barndoor', '0.045214857906103134']
Sam Smith, Kim Petras - Unholy (Official Music Video)-Uq9gPaIzbe8.mp4
91
['barndoor', '0.05029477179